In [ ]:
"""
======================================================
Añade tres variantes de *feature map* pensadas para captar la ligera curvatura
sin caer en barren plateaus:

1. **ZFeatureMap‑α (scale = 0.5)** – rota la mitad de amplitud → evita
   saturar el coseno.
2. **ZFeatureMap‑LinEnt** – ZFeatureMap (reps 1) seguido de cadena CX
   lineal ⟶ introduce correlaciones \(x_i x_{i+1}\) con profundidad mínima.
3. **ZFeatureMap‑α‑LinEnt** – combinación de 1 + 2.

Se comparan con las anteriores `ZFeatureMap-reps1/2` (baseline).

Para mantener tiempos bajos se usa **2 splits** en QSVM (suficiente p/ ver la
tendencia) y búsqueda de `C` limitada a {0.1, 1, 10}.
"""

import warnings, time, numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import ZFeatureMap
from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

warnings.filterwarnings("ignore")
SEED = 12345
np.random.seed(SEED)

sampler = StatevectorSampler()
fidelity = ComputeUncompute(sampler=sampler)

# --------------------- Dataset  -------------------
def load_dataset():
    X_tr, y_tr, X_ts, y_ts = ad_hoc_data(
        training_size=TRAIN_SIZE,
        test_size=TEST_SIZE,
        n=FEATURE_DIM,
        gap=0.3,
        one_hot=False,
        include_sample_total=False,
    )
    y_tr[y_tr == 0] = -1
    y_ts[y_ts == 0] = -1
    return X_tr, y_tr, X_ts, y_ts


# --------------------- Classical baselines  ---------------------

def classical_grids():
    gammas = np.logspace(-3, 1, 5)
    return {
        "LogReg": (LogisticRegression(max_iter=2000), {"C": [0.1, 1, 10]}),
        "SVM-linear": (SVC(kernel="linear"), {"C": [0.1, 1, 10]}),
        "SVM-poly": (SVC(kernel="poly"), {"degree": [3, 4, 5], "C": [0.1, 1, 10], "coef0": [0, 1]}),
        "SVM-rbf": (SVC(kernel="rbf"), {"C": [0.1, 1, 10], "gamma": gammas}),
    }

def eval_classical(X, y):
    scaler = StandardScaler(); Xs = scaler.fit_transform(X)
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.3, random_state=SEED)
    rows=[]
    for split,(tr,ts) in enumerate(sss.split(Xs,y),1):
        Xtr,Xts, ytr,yts = Xs[tr],Xs[ts], y[tr],y[ts]
        for m,(clf,grid) in classical_grids().items():
            gs = GridSearchCV(clf,grid,cv=3,scoring='accuracy',n_jobs=-1)
            t0=time.time(); gs.fit(Xtr,ytr); dur=time.time()-t0
            pred=gs.best_estimator_.predict(Xts)
            rows.append({"split":split,"model":m,"acc":accuracy_score(yts,pred),"f1":f1_score(yts,pred),"time":dur})
    return pd.DataFrame(rows)

# --------------------- Quantum feature‑maps -------------------------------
FEATURE_DIM=4

# Helper to build linear‑CX chain
def add_linear_entanglement(circ:QuantumCircuit):
    for i in range(circ.num_qubits-1):
        circ.cx(i,i+1)
    return circ

def build_maps():
    maps={}
    # baseline ZFeatureMap 1 & 2 reps
    maps["ZF-reps1"] = (ZFeatureMap(FEATURE_DIM,reps=1), 1.0)
    maps["ZF-reps2"] = (ZFeatureMap(FEATURE_DIM,reps=2), 1.0)
    # scaled θ via data factor 0.5
    maps["ZF-alpha05"] = (ZFeatureMap(FEATURE_DIM,reps=1), 0.5)
    # lin entanglement
    circ_lin = ZFeatureMap(FEATURE_DIM,reps=1)
    maps["ZF-LinEnt"] = (add_linear_entanglement(circ_lin), 1.0)
    # scaled + entanglement
    circ_lin_a = ZFeatureMap(FEATURE_DIM,reps=1)
    maps["ZF-alpha05-LinEnt"] = (add_linear_entanglement(circ_lin_a), 0.5)
    return maps

# --------------------- QSVM eval ------------------------------------------

def eval_qsvm(X,y):
    scaler=MinMaxScaler((0,np.pi)); Xs=scaler.fit_transform(X)
    sss=StratifiedShuffleSplit(n_splits=10,test_size=0.3,random_state=SEED)
    out=[]
    for name,(fmap,data_scale) in build_maps().items():
        accs,f1s=[],[]
        for split,(tr,ts) in enumerate(sss.split(Xs,y),1):
            Xtr_raw,Xts_raw,ytr,yts = Xs[tr],Xs[ts],y[tr],y[ts]
            # apply optional scaling factor to rotation angles
            Xtr = Xtr_raw * data_scale
            Xts = Xts_raw * data_scale
            qk=FidelityQuantumKernel(feature_map=fmap,fidelity=fidelity)
            #print(fmap.decompose())
            Ktr=qk.evaluate(Xtr); Kts=qk.evaluate(Xts,Xtr)
            bestC, maxAcc = None, 0
            for C in [0.1,1,10]:
                skf=StratifiedKFold(n_splits=10,shuffle=True,random_state=SEED)
                acc_fold=[]
                for tri,vali in skf.split(Ktr,ytr):
                    svc=SVC(kernel='precomputed',C=C); svc.fit(Ktr[tri][:,tri],ytr[tri])
                    acc_fold.append(accuracy_score(ytr[vali],svc.predict(Ktr[vali][:,tri])))
                m=np.mean(acc_fold)
                if m > maxAcc:
                    maxAcc = m
                    bestC = C
            svc=SVC(kernel='precomputed',C=bestC); svc.fit(Ktr,ytr)
            pred=svc.predict(Kts)
            accs.append(accuracy_score(yts,pred)); f1s.append(f1_score(yts,pred))
        out.append({"map":name,"acc_mean":np.mean(accs),"acc_std":np.std(accs),"f1_mean":np.mean(f1s)})
    return pd.DataFrame(out)

# ----------------------------- MAIN ---------------------------------------
if __name__=='__main__':
    print('⏳ Evaluando clásicos...')
    df_c=eval_classical(X_raw,y_raw)
    print(df_c.groupby('model')[['acc','f1']].agg(['mean','std']))

    print('\n⏳ Evaluando Kernles Cuanticos con feature‑maps mejorados...')
    df_q=eval_qsvm(X_raw,y_raw)
    print(df_q.sort_values('acc_mean',ascending=False))

    with pd.ExcelWriter('Resultados_IrisLineal_QBoost.xlsx') as w:
        df_c.to_excel(w,sheet_name='Clasicos',index=False)
        df_q.to_excel(w,sheet_name='Cuanticos',index=False)
    print('\n Resultados guardados en Resultados_IrisLineal_QBoost.xlsx')
